In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 932.9/932.9 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 87.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 62.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [ ]:
!python -c 'from torchvision.models.detection import maskrcnn_resnet50_fpn, MaskRCNN_ResNet50_FPN_Weights; maskrcnn_resnet50_fpn(weights=MaskRCNN_ResNet50_FPN_Weights.DEFAULT)'

Downloading: "https://download.pytorch.org/models/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth" to /root/.cache/torch/hub/checkpoints/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth
100% 170M/170M [00:02<00:00, 85.4MB/s]


In [ ]:
from ultralytics import YOLO
model = YOLO("yolo11n-pose.pt")
import torch
import numpy as np
import torchvision
import torchvision.models.detection as detection
import os
import json
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as Data
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, precision_score, recall_score, f1_score
import cv2
import time
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor
from sklearn.model_selection import KFold
from scipy.ndimage import shift
from joblib import Parallel, delayed
from sklearn.model_selection import LeaveOneGroupOut
import joblib

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 5.97M/5.97M [00:00<00:00, 64.2MB/s]


In [ ]:

# MIDDLE_FRAMES_NO = 120
# Define the specific indexes of keypoints to retain
specific_indexes = [0, 5, 6, 7, 8, 9, 10]

# Function to process videos and store tracked keypoints in JSON files
def process_videos(video_dir, json_dir):
    # Ensure the JSON directory exists
    os.makedirs(json_dir, exist_ok=True)

    # Loop through all video files in the provided directory
    for video_filename in os.listdir(video_dir):
        video_path = os.path.join(video_dir, video_filename)
        # Save the tracked keypoints to a JSON file with the same name as the video
        json_filename = os.path.splitext(video_filename)[0] + ".json"
        json_path = os.path.join(json_dir, json_filename)

        if os.path.exists(json_path):
            continue

        # Skip non-video files
        if not video_filename.lower().endswith(('.mp4', '.mov', '.avi')):
            continue

        print("processing video ", video_path)
        # Open the video file
        cap = cv2.VideoCapture(video_path)
        vid_tracked_keypoints = {}  # Dictionary to store tracked keypoints per frame

        # Get the total number of frames in the video
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        # Create a new tracker instance for this video
        model = YOLO("/content/yolo11n-pose.pt")  # Load the model again for each video

        # Loop through the middle 70 frames
        for frame_num in range(0, total_frames):
            cap.set(cv2.CAP_PROP_POS_FRAMES, frame_num)
            success, frame = cap.read()

            if not success:
                break

            # Run YOLOv11 tracking on the frame
            results = model.track(frame, persist=True, max_det=2, classes=[0], verbose=False, device="cpu")

            # Extract keypoints and track IDs from the results
            keypoints = results[0].keypoints.data
            track_ids = results[0].boxes.id.int().cpu().tolist()

            # Initialize dictionary to store the tracked keypoints for this frame
            frame_tracked_keypoints = {}

            # Ensure there are exactly 2 people detected in the frame
            if len(track_ids) != 2:
                raise ValueError(f"Expected 2 persons, but found {len(track_ids)} persons in frame {frame_num}.")

            # Iterate over each person's keypoints
            for person_data, track in zip(keypoints, track_ids):
                person_keypoints = []
                for index in specific_indexes:
                    x, y, conf = person_data[index].tolist()
                    # if conf >= 0.25:
                    person_keypoints.append([x, y])  # Add valid (x, y) points
                    # else:
                    #     person_keypoints.append([0, 0])  # Add (0, 0) for low confidence keypoints

                # Store the tracked keypoints for this person
                frame_tracked_keypoints[track] = person_keypoints

            # Store the tracked keypoints for this frame
            vid_tracked_keypoints[frame_num] = frame_tracked_keypoints

        with open(json_path, 'w') as json_file:
            json.dump(vid_tracked_keypoints, json_file, indent=4)

        # Release the video capture object
        cap.release()

    print("Video processing complete!")

# Example usage
video_dir = '/content/drive/MyDrive/FYP/Dataset'  # Replace with the path where videos are stored
json_dir = '/content/drive/MyDrive/FYP/keypoints'  # Replace with the path where JSON files will be saved
process_videos(video_dir, json_dir)


processing video  /content/drive/MyDrive/FYP/Dataset/52_1.mp4
processing video  /content/drive/MyDrive/FYP/Dataset/53_3.mp4
processing video  /content/drive/MyDrive/FYP/Dataset/54_2.mp4
processing video  /content/drive/MyDrive/FYP/Dataset/55_1.mp4
processing video  /content/drive/MyDrive/FYP/Dataset/57_3.mp4
processing video  /content/drive/MyDrive/FYP/Dataset/56_2.mp4
processing video  /content/drive/MyDrive/FYP/Dataset/58_1.mp4
processing video  /content/drive/MyDrive/FYP/Dataset/59_3.mp4
processing video  /content/drive/MyDrive/FYP/Dataset/60_2.mp4
processing video  /content/drive/MyDrive/FYP/Dataset/61_1.mp4
processing video  /content/drive/MyDrive/FYP/Dataset/62_3.mp4
processing video  /content/drive/MyDrive/FYP/Dataset/63_2.mp4
processing video  /content/drive/MyDrive/FYP/Dataset/64_1.mp4
processing video  /content/drive/MyDrive/FYP/Dataset/65_3.mp4
processing video  /content/drive/MyDrive/FYP/Dataset/67_1.mp4
processing video  /content/drive/MyDrive/FYP/Dataset/68_3.mp4
processi

In [ ]:

# Function to get ground truth data

# Data directories
DATA_DIR = '/content/drive/MyDrive/FYP/Dataset'
KEYPOINTS_DIR = '/content/drive/MyDrive/FYP/keypoints'
SILHOUETES_DIR = '/content/drive/MyDrive/FYP/Silhouetes'
HOF_DIR = '/content/drive/MyDrive/FYP/HOF'
FIXED_HOF_DIR = '/content/drive/MyDrive/FYP/Fixed_HOF'
DIST_DIR = '/content/drive/MyDrive/FYP/Dist'
ANGLE_DIR = '/content/drive/MyDrive/FYP/Angle'
LTP_DIR = '/content/drive/MyDrive/FYP/LTP'
VELOCITY_DIR = '/content/drive/MyDrive/FYP/Velocity'
FIXED_VELOCITY_DIR = '/content/drive/MyDrive/FYP/Fixed_velocity'

os.makedirs(KEYPOINTS_DIR, exist_ok=True)
os.makedirs(SILHOUETES_DIR, exist_ok=True)
os.makedirs(HOF_DIR, exist_ok=True)
os.makedirs(FIXED_HOF_DIR, exist_ok=True)
os.makedirs(DIST_DIR, exist_ok=True)
os.makedirs(ANGLE_DIR, exist_ok=True)
os.makedirs(LTP_DIR, exist_ok=True)
os.makedirs(VELOCITY_DIR, exist_ok=True)
os.makedirs(FIXED_VELOCITY_DIR, exist_ok=True)

NUM_FOLDS = 3

# Function to get ground truth data
def get_ground_truth(num_folds=NUM_FOLDS, random_state=42):
    video_path_lst, video_lst, action_lst, keypoints_lst, silhouettes_lst, hof_lst, dist_lst, angle_lst = [], [], [], [], [], [], [], []

    ltp_lst, vel_lst, fixed_vel_lst, fixed_hof_lst = [], [], [], []

    for video in os.listdir(DATA_DIR):
        video_name, _ = os.path.splitext(video)
        video_id, action = video_name.split('_')

        keypoints_path = os.path.join(KEYPOINTS_DIR, f'{video_name}.json')
        hof_path = os.path.join(HOF_DIR, f'{video_name}.json')
        dist_path = os.path.join(DIST_DIR, f'{video_name}.json')
        angle_path = os.path.join(ANGLE_DIR, f'{video_name}.json')
        ltp_path = os.path.join(LTP_DIR, f'{video_name}.json')
        vel_path = os.path.join(VELOCITY_DIR, f'{video_name}.json')
        fixed_vel_path = os.path.join(FIXED_VELOCITY_DIR, f'{video_name}.json')
        fixed_hof_path = os.path.join(FIXED_HOF_DIR, f'{video_name}.json')

        silhouettes_path = os.path.join(SILHOUETES_DIR, video_name)
        video_path = os.path.join(DATA_DIR, video)

        video_lst.append(video_name)
        action_lst.append(int(action) - 1)
        keypoints_lst.append(keypoints_path)
        silhouettes_lst.append(silhouettes_path)
        video_path_lst.append(video_path)
        hof_lst.append(hof_path)
        dist_lst.append(dist_path)
        angle_lst.append(angle_path)
        ltp_lst.append(ltp_path)
        vel_lst.append(vel_path)
        fixed_vel_lst.append(fixed_vel_path)
        fixed_hof_lst.append(fixed_hof_path)

    dataframe_dict = {'video_id': video_lst,
                      'action': action_lst,
                      'keypoints_path': keypoints_lst,
                      'silhouettes_path': silhouettes_lst,
                      'video_path': video_path_lst,
                      'hof_path': hof_lst,
                      'dist_path': dist_lst,
                      'angle_path': angle_lst,
                      'ltp_path': ltp_lst,
                      'vel_path': vel_lst,
                      'fixed_vel_path': fixed_vel_lst,
                      'fixed_hof_path': fixed_hof_lst}

    ground_truth = pd.DataFrame(dataframe_dict)
    ground_truth = ground_truth.set_index('video_id')

    # K-Fold
    skf = KFold(n_splits=num_folds, shuffle=True, random_state=random_state)
    ground_truth['fold'] = -1
    X = ground_truth.index.values
    y = ground_truth['action'].values

    for fold_idx, (_, val_idx) in enumerate(skf.split(X, y)):
        ground_truth.loc[ground_truth.index[val_idx], 'fold'] = fold_idx

    return ground_truth

gt = get_ground_truth()
print(gt)


          action                                  keypoints_path  \
video_id                                                           
1_2            1   /content/drive/MyDrive/FYP/keypoints/1_2.json   
2_2            1   /content/drive/MyDrive/FYP/keypoints/2_2.json   
3_2            1   /content/drive/MyDrive/FYP/keypoints/3_2.json   
4_2            1   /content/drive/MyDrive/FYP/keypoints/4_2.json   
16_1           0  /content/drive/MyDrive/FYP/keypoints/16_1.json   
...          ...                                             ...   
90_2           1  /content/drive/MyDrive/FYP/keypoints/90_2.json   
91_1           0  /content/drive/MyDrive/FYP/keypoints/91_1.json   
92_3           2  /content/drive/MyDrive/FYP/keypoints/92_3.json   
93_2           1  /content/drive/MyDrive/FYP/keypoints/93_2.json   
94_1           0  /content/drive/MyDrive/FYP/keypoints/94_1.json   

                                    silhouettes_path  \
video_id                                               
1_2

In [ ]:
# segmentation

def silhouette_extraction(frame, prediction):
    new_width = 480
    new_height = 320
    # frame = cv2.imread(frame_path)
    # plt.imshow(frame)
    # plt.show()
    resized_frame = cv2.resize(frame, (new_width, new_height))
    gray_image = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2GRAY)
    masks_image = np.zeros_like(gray_image, dtype=np.uint8)

    masks = prediction['masks']

    # Loop through masks, assuming person class index is 1 (adjust if needed
    for i in range(masks.shape[0]):
        if prediction['labels'][i] == 1 and prediction['scores'][i] > 0.5:  # Adjust class index for person

            mask = masks[i].cpu().numpy().squeeze()

            # plt.imshow(frame_copy)
            # plt.title('frame copy')
            # plt.show()
            masks_image[mask > 0.4] = 1

    gray_image = gray_image * masks_image

    # white_gray_image = gray_image
    # white_gray_image[white_gray_image == 0] = 255

    # plt.imshow(gray_image, cmap="gray")
    # plt.title('gray image')
    # plt.show()

    return gray_image

def process_frames(video_path, silhouette_path):
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    selected_frames = 61
    half_frames = total_frames // 2
    middle_starting_frame = max(0, min(half_frames - selected_frames // 2, total_frames - selected_frames))
    middle_ending_frame = middle_starting_frame + selected_frames
    batch_size = 8
    silhouette_paths = []
    tensor_frames = []
    frames = []
    j = 0

    # Read frames from the video
    for frame_no in range(middle_starting_frame, middle_ending_frame):
        # Set the frame position
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_no)

        # Read the current frame from the video
        ret, frame = cap.read()

        if not ret:
            raise ValueError(f"Failed to read frame {frame_no} from the video!")

        # Set frame and silhouette paths
        silhouettes_path = os.path.join(silhouette_path, f"{frame_no}.png")

        # Check if silhouette exists already
        if os.path.exists(silhouettes_path):
            print(f"Silhouette {silhouettes_path} already exists")
        else:
            silhouette_paths.append(silhouettes_path)
            frames.append(frame)  # Adding the frame name to paths (not using full path for video frames)

            # Resize the frame (480x320) and convert it to RGB
            new_width = 480
            new_height = 320
            resized_frame = cv2.resize(frame, (new_width, new_height))
            resized_frame = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2RGB)

            # Transform to tensor
            transform = torchvision.transforms.ToTensor()
            resized_frame = transform(resized_frame).unsqueeze(0)  # Add batch dimension

            # Append the tensor to the list
            tensor_frames.append(resized_frame)

    # Release the video capture object
    cap.release()
    # print("silhouette path", silhouette_paths)
    batches = []

    for i in range(0, len(tensor_frames), batch_size):
        batch = tensor_frames[i:i+batch_size]
        batch = torch.cat(batch, dim=0)
        # batches.append(batch)
        batches.append(batch.to(device))  # Move batch to GPU

    for batch_no, batch in enumerate(batches):
        with torch.no_grad():
          predictions = model(batch)

        # Iterate over predictions and corresponding frame information
        for i in range(len(batch)):
          prediction = predictions[i]  # Access prediction for the current frame

          # Extract silhouettes
          silhouette = silhouette_extraction(frames[j], prediction)

          # Save silhouettes
          cv2.imwrite(silhouette_paths[j], silhouette)
          print(f"Processed frame: {silhouette_paths[j]}")

          # plt.imshow(silhouette)
          # plt.show()

          j = j + 1

if __name__ == "__main__":
    # Load a pre-trained Mask R-CNN model
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(weights=detection.MaskRCNN_ResNet50_FPN_Weights.DEFAULT)
    model.to(device)  # Move the model to GPU
    model.eval()

    video_df = get_ground_truth()
    # print(video_df)
    for video, video_row in video_df.iterrows():

      process_frames(video_row['video_path'], video_row['silhouettes_path'])

In [ ]:

import json

def extract_hof_features(prev_frame, curr_frame):
    # Calculate the dense optical flow using Farneback method
    flow = cv2.calcOpticalFlowFarneback(prev_frame, curr_frame, None,
                                        pyr_scale=0.5, levels=3, winsize=15,
                                        iterations=3, poly_n=5, poly_sigma=1.2,
                                        flags=0)

    # Compute magnitude and angle of the flow vectors
    magnitude, angle = cv2.cartToPolar(flow[..., 0], flow[..., 1], angleInDegrees=True)

    # Bin the angles into 8 bins
    num_bins = 18
    bin_edges = np.linspace(0, 360, num_bins + 1)
    hof_features = np.histogram(angle, bins=bin_edges, weights=magnitude, density=True)[0]

     # Avoid NaNs by replacing them with zeros
    hof_features[np.isnan(hof_features)] = 0
    return hof_features

def process_frames_for_hof(video_silhouettes_path, target_hof_features_path):

    silhouettes = os.listdir(video_silhouettes_path)
    sorted_silhouettes = sorted(silhouettes, key=lambda x: int(x.split('.')[0]))
    sorted_silhouettes = sorted_silhouettes[1:]

    hof_features_list = []

    for i in range(1, len(sorted_silhouettes)):  # Start from the second frame

        curr_silhouette_path = os.path.join(video_silhouettes_path, sorted_silhouettes[i])
        prev_silhouette_path = os.path.join(video_silhouettes_path, sorted_silhouettes[i-1])

        curr_silhouette = cv2.imread(curr_silhouette_path, cv2.IMREAD_GRAYSCALE)
        prev_silhouette = cv2.imread(prev_silhouette_path, cv2.IMREAD_GRAYSCALE)

        # Extract HOF features between current and previous silhouette
        hof_features = extract_hof_features(prev_silhouette, curr_silhouette)

        hof_features_list.append(hof_features)

    hof_features_list = np.array(hof_features_list)

    hof_features_list = np.pad(hof_features_list, ((1, 0), (0, 0)), mode='constant', constant_values=0)
    print(hof_features_list.shape)

    if hof_features_list.shape == (0, ):
        raise ValueError("No HOF features extracted")


    with open(target_hof_features_path, 'w') as f:
        hof_features_list = hof_features_list.tolist()
        json.dump({"hof": hof_features_list}, f)

if __name__ == "__main__":

    video_df = get_ground_truth()

    for video, video_row in video_df.iterrows():

        print("video no" , video)

        # if os.path.exists(video_row['hof_path']):
        #     print(f"hof features file {video_row['hof_path']} exists already")
        #     continue

        process_frames_for_hof(video_row['silhouettes_path'], video_row['fixed_hof_path'])



video no 1_2
(60, 18)
video no 2_2
(60, 18)
video no 3_2
(60, 18)
video no 4_2
(60, 18)
video no 16_1
(60, 18)
video no 15_1
(60, 18)
video no 14_1
(60, 18)
video no 13_1
(60, 18)
video no 12_3
(60, 18)
video no 11_3
(60, 18)
video no 9_3
(60, 18)
video no 7_3
(60, 18)
video no 17_1
(60, 18)
video no 18_2
(60, 18)
video no 19_3
(60, 18)
video no 20_1
(60, 18)
video no 23_1
(60, 18)
video no 26_1
(60, 18)
video no 29_1
(60, 18)
video no 32_2
(60, 18)
video no 33_1
(60, 18)
video no 34_3
(60, 18)
video no 35_2
(60, 18)
video no 36_1
(60, 18)
video no 37_3
(60, 18)
video no 38_2
(60, 18)
video no 40_3
(60, 18)
video no 39_1
(60, 18)
video no 41_3
(60, 18)
video no 43_2
(60, 18)
video no 42_1
(60, 18)
video no 44_3
(60, 18)
video no 45_1
(60, 18)
video no 46_2
(60, 18)
video no 22_3
(60, 18)
video no 21_2
(60, 18)
video no 24_2
(60, 18)
video no 25_3
(60, 18)
video no 27_2
(60, 18)
video no 28_3
(60, 18)
video no 30_2
(60, 18)
video no 31_3
(60, 18)
video no 47_3
(60, 18)
video no 48_2
(60

In [ ]:
# parallel ltp


def compute_ltp(image, radius=1, neighbors=8, threshold=5):
    """
    Compute Local Ternary Patterns (LTP) for a grayscale image efficiently.
    Uses NumPy vectorized operations and SciPy shift to improve speed.
    """
    image = image.astype(np.float32)
    padded_image = cv2.copyMakeBorder(image, radius, radius, radius, radius, cv2.BORDER_REFLECT)
    # padded_image = np.pad(image, pad_width=radius, mode='edge')

    h, w = image.shape
    ltp_pos = np.zeros((h, w), dtype=np.uint8)
    ltp_neg = np.zeros((h, w), dtype=np.uint8)

    offsets = [
        (int(np.round(radius * np.sin(2 * np.pi * n / neighbors))),
         int(np.round(radius * np.cos(2 * np.pi * n / neighbors))))
        for n in range(neighbors)
    ]

    # Create stacked neighbor shifts
    neighbors_matrix = np.stack([
        shift(padded_image, shift=(dy, dx), mode='nearest')[radius:-radius, radius:-radius]
        for dy, dx in offsets
    ], axis=-1)  # Shape: (h, w, neighbors)

    center_matrix = image[..., None]  # Shape: (h, w, 1)
    diff = neighbors_matrix - center_matrix  # Compute differences

    # Compute LTP codes using NumPy boolean indexing (Vectorized)
    ltp_pos = np.sum((diff > threshold) * (1 << np.arange(neighbors)), axis=-1, dtype=np.uint8)
    ltp_neg = np.sum((diff < -threshold) * (1 << np.arange(neighbors)), axis=-1, dtype=np.uint8)

    return ltp_pos, ltp_neg


def extract_ltp_feat_for_frame(image, radius=1, neighbors=8, threshold=5):
    """
    Extract LTP histogram features efficiently.
    """
    ltp_pos, ltp_neg = compute_ltp(image, radius, neighbors, threshold)

    # Compute histograms efficiently
    pos_hist = np.bincount(ltp_pos.ravel(), minlength=256)
    neg_hist = np.bincount(ltp_neg.ravel(), minlength=256)

    # Concatenate histograms and convert to float32
    return np.concatenate((pos_hist, neg_hist)).astype(np.float32)


def ltp_feat_extraction(video_silhouettes_path, target_ltp_features_path):
    """
    Process silhouette frames and extract LTP features in parallel.
    """

    silhouettes = os.listdir(video_silhouettes_path)
    silhouettes = sorted(silhouettes, key=lambda x: int(x.split('.')[0]))
    silhouettes = silhouettes[1:]
    # Use Joblib for parallel processing
    ltp_features = Parallel(n_jobs=-1)(
        delayed(extract_ltp_feat_for_frame)(
            cv2.imread(os.path.join(video_silhouettes_path, silhouette_path), cv2.IMREAD_GRAYSCALE)
            ) for silhouette_path in silhouettes
    )

    ltp_features = np.array(ltp_features)

    print(ltp_features.shape)
    if ltp_features.shape == (0, ):
        raise ValueError("No LTP features extracted")

    with open(target_ltp_features_path, 'w') as f:
        json.dump({"ltp": ltp_features.tolist()}, f)


if __name__ == "__main__":

    video_df = get_ground_truth()  # Ensure this function provides video data paths and output locations

    for video, video_row in video_df.iterrows():
        print("Processing video no:", video)

        # if os.path.exists(video_row['ltp_path']):
        #     print(f"LTP features file {video_row['ltp_path']} already exists")
        #     continue

        ltp_feat_extraction(video_row['silhouettes_path'], video_row['ltp_path'])



Processing video no: 1_2
(60, 512)
Processing video no: 2_2
(60, 512)
Processing video no: 3_2
(60, 512)
Processing video no: 4_2
(60, 512)
Processing video no: 16_1
(60, 512)
Processing video no: 15_1
(60, 512)
Processing video no: 14_1
(60, 512)
Processing video no: 13_1
(60, 512)
Processing video no: 12_3
(60, 512)
Processing video no: 11_3
(60, 512)
Processing video no: 9_3
(60, 512)
Processing video no: 7_3
(60, 512)
Processing video no: 17_1
(60, 512)
Processing video no: 18_2
(60, 512)
Processing video no: 19_3
(60, 512)
Processing video no: 20_1
(60, 512)
Processing video no: 23_1
(60, 512)
Processing video no: 26_1
(60, 512)
Processing video no: 29_1
(60, 512)
Processing video no: 32_2
(60, 512)
Processing video no: 33_1
(60, 512)
Processing video no: 34_3
(60, 512)
Processing video no: 35_2
(60, 512)
Processing video no: 36_1
(60, 512)
Processing video no: 37_3
(60, 512)
Processing video no: 38_2
(60, 512)
Processing video no: 40_3
(60, 512)
Processing video no: 39_1
(60, 512

In [ ]:
# intra and inter distance changes

# Indices and mapping as provided
indices = {
    'N1': 0, 'LS1': 1, 'RS1': 2, 'LE1': 3, 'RE1': 4,
    'LW1': 5, 'RW1': 6,
    'N2': 0, 'LS2': 1, 'RS2': 2, 'LE2': 3, 'RE2': 4,
    'LW2': 5, 'RW2': 6,
}

intra_distance_mapping = {
    "N1": ["RW1"],
    "LS1": ["LW1"],
    "RE1": ["RW1"],
    "LE1": ["LW1"],

     "N2": ["RW2"],
    "LS2": ["LW2"],
    "RE2": ["RW2"],
    "LE2": ["LW2"],
}

inter_distance_mapping = {

    "LS1": ["RS2"],
    "LW1": ["RW2"],
    "LE1": ["RE2"],
    "N1": ["N2"],
}

# Function to compute distances
def compute_distance_changes(input_file, output_file):
    # Load keypoints from JSON
    with open(input_file, 'r') as f:
        keypoints = json.load(f)

    sorted_frames = sorted(keypoints.keys(), key=lambda x: int(x))

    total_frames = len(sorted_frames)

    # Calculate the start and end indices for the middle 60 frames
    start_index = max((total_frames - 60) // 2, 0)  # Ensure index is non-negative
    end_index = start_index + 60

    # Select the middle 60 frames
    sorted_frames = sorted_frames[start_index:end_index]

    intra_inter_distances = []

    for frame in sorted_frames:
        coords = keypoints[frame]
        # Identify left and right persons based on x-coordinates of their noses
        left_person_id, right_person_id = sorted(
            coords.keys(),
            key=lambda pid: coords[pid][0][0]
        )

        frame_intra_distances = []
        frame_inter_distances = []

        # Iterate through intra-person distances
        for joint_name, target_joints in intra_distance_mapping.items():
            if joint_name[-1] == "1":
                person_id = left_person_id
            else:
                person_id = right_person_id

            joint_idx = indices[joint_name]
            for target_joint in target_joints:
                target_joint_idx = indices[target_joint]
                try:
                    euclidean_distance = np.linalg.norm(
                        np.array(coords[person_id][joint_idx]) -
                        np.array(coords[person_id][target_joint_idx])
                    )
                    frame_intra_distances.append(euclidean_distance)
                except IndexError:
                    print("Error")
                    continue

        # Iterate through inter-person distances
        for joint_name, target_joints in inter_distance_mapping.items():
            if joint_name[-1] == "1":
                joint_person_id = left_person_id
            else:
                joint_person_id = right_person_id

            joint_idx = indices[joint_name]
            for target_joint in target_joints:
                if target_joint[-1] == "1":
                    target_person_id = left_person_id
                else:
                    target_person_id = right_person_id

                target_joint_idx = indices[target_joint]
                try:
                    euclidean_distance = np.linalg.norm(
                        np.array(coords[joint_person_id][joint_idx]) -
                        np.array(coords[target_person_id][target_joint_idx])
                    )
                    frame_inter_distances.append(euclidean_distance)
                except IndexError:
                    print("Error")
                    print(joint_person_id)
                    print(target_person_id)

        combined_distances = frame_intra_distances + frame_inter_distances
        intra_inter_distances.append(combined_distances)

    intra_inter_distances = np.array(intra_inter_distances)

    print("shape", intra_inter_distances.shape)
    # Save results to output JSON
    with open(output_file, 'w') as f:
        json.dump({"intra_inter_distance_mapping": intra_inter_distances.tolist()}, f)


if __name__ == "__main__":
    video_df = get_ground_truth()

    for video, video_row in video_df.iterrows():

        print("video no" , video)

        # if os.path.exists(video_row['dist']):
        #     print(f"distance change file {video_row['dist']} exists already")
        #     continue

        compute_distance_changes(video_row['keypoints'], video_row['dist'])


In [ ]:

# Indices and mapping as provided
indices = {
    'N1': 0, 'LS1': 1, 'RS1': 2, 'LE1': 3, 'RE1': 4,
    'LW1': 5, 'RW1': 6,
    'N2': 0, 'LS2': 1, 'RS2': 2, 'LE2': 3, 'RE2': 4,
    'LW2': 5, 'RW2': 6,
}

intra_angle_mapping = {
    "N1": ["RS1", "LS1"],
    "RS1": ["RE1"],
    "LS1": ["LE1"],
    "RE1": ["RW1"],
    "LE1": ["LW1"],

    "N2": ["RS2", "LS2"],
    "RS2": ["RE2"],
    "LS2": ["LE2"],
    "RE2": ["RW2"],
    "LE2": ["LW2"]

}

inter_angle_mapping = {
    "LW1": ["RW2"],
    "LE1": ["RE2"],
    "LS1": ["RS1"]
}


# Function to compute angles
def compute_angle_changes(input_file, output_file):
    # Load keypoints from JSON
    with open(input_file, 'r') as f:
        keypoints = json.load(f)

    sorted_frames = sorted(keypoints.keys(), key=lambda x: int(x))

    total_frames = len(sorted_frames)

    # Calculate the start and end indices for the middle 60 frames
    start_index = max((total_frames - 60) // 2, 0)  # Ensure index is non-negative
    end_index = start_index + 60

    # Select the middle 60 frames
    sorted_frames = sorted_frames[start_index:end_index]

    intra_inter_angles = []

    for frame in sorted_frames:
        coords = keypoints[frame]
        # Identify left and right persons based on x-coordinates of their noses
        left_person_id, right_person_id = sorted(
            coords.keys(),
            key=lambda pid: coords[pid][0][0]
        )

        frame_intra_angles = []
        frame_inter_angles = []

        # Iterate through intra-person angles
        for joint_name, target_joints in intra_angle_mapping.items():
            if joint_name[-1] == "1":
                person_id = left_person_id
            else:
                person_id = right_person_id

            joint_idx = indices[joint_name]
            for target_joint in target_joints:
                target_joint_idx = indices[target_joint]
                try:
                    delta_x = coords[person_id][target_joint_idx][0] - coords[person_id][joint_idx][0]
                    delta_y = coords[person_id][target_joint_idx][1] - coords[person_id][joint_idx][1]
                    angle = np.arctan2(delta_y, delta_x)
                    frame_intra_angles.append(angle)
                except IndexError:
                    print(f"Error computing intra angle for joint {joint_name}")
                    continue

        # Iterate through inter-person angles
        for joint_name, target_joints in inter_angle_mapping.items():
            if joint_name[-1] == "1":
                joint_person_id = left_person_id
            else:
                joint_person_id = right_person_id

            joint_idx = indices[joint_name]
            for target_joint in target_joints:
                if target_joint[-1] == "1":
                    target_person_id = left_person_id
                else:
                    target_person_id = right_person_id

                target_joint_idx = indices[target_joint]
                try:
                    delta_x = coords[target_person_id][target_joint_idx][0] - coords[joint_person_id][joint_idx][0]
                    delta_y = coords[target_person_id][target_joint_idx][1] - coords[joint_person_id][joint_idx][1]
                    angle = np.arctan2(delta_y, delta_x)
                    frame_inter_angles.append(angle)
                except IndexError:
                    print(f"Error computing inter angle between {joint_name} and {target_joint}")
                    continue

        combined_angles = frame_intra_angles + frame_inter_angles
        intra_inter_angles.append(combined_angles)

    intra_inter_angles = np.array(intra_inter_angles)

    print("shape", intra_inter_angles.shape)
    # Save results to output JSON
    with open(output_file, 'w') as f:
        json.dump({"intra_inter_angle_mapping": intra_inter_angles.tolist()}, f)


if __name__ == "__main__":
    video_df = get_ground_truth()

    for video, video_row in video_df.iterrows():
        print("video no", video)

        if os.path.exists(video_row['angle']):
            print(f"angle change file {video_row['angles']} exists already")
            continue

        compute_angle_changes(video_row['keypoints'], video_row['angle'])


In [ ]:

# Keypoint indices (for reference)
# 0: Nose, 1: Left Shoulder, 2: Right Shoulder, 3: Left Elbow, 4: Right Elbow,
# 5: Left Wrist, 6: Right Wrist, 7: Left Hip, 8: Right Hip, 9: Left Knee,
# 10: Right Knee, 11: Left Ankle, 12: Right Ankle

def compute_velocity_features(input_file, output_file):
    # Load keypoints from JSON
    with open(input_file, 'r') as f:
        keypoints_data = json.load(f)  # Load the nested structure

    # Convert the nested structure into a NumPy array
    frames = sorted(keypoints_data.keys(), key=lambda x: int(x))  # Sort frames
    persons = sorted(keypoints_data[frames[0]].keys(), key=lambda x: int(x))  # Sort persons
    num_frames = len(frames)
    num_persons = len(persons)
    num_keypoints = len(keypoints_data[frames[0]][persons[0]])  # Number of keypoints per person

    # Initialize a NumPy array to store keypoints
    keypoints = np.zeros((num_frames, num_persons, num_keypoints, 2))  # (frames, persons, keypoints, coordinates)

    # Populate the keypoints array
    for frame_idx, frame in enumerate(frames):
        for person_idx, person in enumerate(persons):
            keypoints[frame_idx, person_idx] = keypoints_data[frame][person]

    # Reorder persons based on the sum of x-coordinates in the first frame
    sum_x_coords = np.sum(keypoints[0, :, :, 0], axis=1)  # Sum of x-coordinates for each person in the first frame
    reorder_indices = np.argsort(sum_x_coords)  # Indices to reorder persons

    # print(reorder_indices)
    # Reorder keypoints array
    keypoints = keypoints[:, reorder_indices, :, :]

    # Extract middle 60 frames
    required_frames = 60

    if num_frames < required_frames:
        raise ValueError("Total frames are less than 60. Cannot extract middle 60 frames.")
        return

    middle_start = (num_frames - required_frames) // 2  # Start index of middle 60 frames
    middle_end = middle_start + required_frames  # End index of middle 60 frames
    keypoints = keypoints[middle_start:middle_end, :, :, :]  # Shape: (60, persons, keypoints, coordinates)

    # Reshape keypoints array to (num_frames, num_persons * num_keypoints, 2)
    keypoints_reshaped = keypoints.reshape(required_frames, num_persons * num_keypoints, 2)

    # Compute velocity features
    if keypoints_reshaped.shape[0] < 2:
        raise ValueError("Not enough frames to compute velocity.")
        return

    # Extract X and Y coordinates for all keypoints
    prev_coords = keypoints_reshaped[:-1, :, :]  # All frames except last
    curr_coords = keypoints_reshaped[1:, :, :]   # All frames except first

    # Compute Euclidean distance for velocity (frame-to-frame displacement)
    velocity = np.linalg.norm(curr_coords - prev_coords, axis=2)  # Shape: (59, num_persons * num_keypoints)

    # velocity_padded = velocity
    # Pad velocity array to match the original number of frames (60)
    velocity_padded = np.pad(velocity, ((1, 0), (0, 0)), mode='constant', constant_values=0)

    print("Velocity feature shape:", velocity_padded.shape)

    # Save results to JSON
    with open(output_file, 'w') as f:
        json.dump({"velocity": velocity_padded.tolist()}, f)

    print("Velocity features computed and saved successfully.")


if __name__ == "__main__":
     video_df = get_ground_truth()

     for video, video_row in video_df.iterrows():
        print("Processing video:", video)

        if os.path.exists(video_row['fixed_velocity_path']):
            print(f"Velocity features file {video_row['velocity_path']} already exists.")
            continue

        compute_velocity_features(video_row['keypoints_path'], video_row['fixed_velocity_path'])


Processing video: 1_2
Velocity features file /content/drive/MyDrive/FYP/Velocity/1_2.json already exists.
Processing video: 2_2
Velocity features file /content/drive/MyDrive/FYP/Velocity/2_2.json already exists.
Processing video: 3_2
Velocity features file /content/drive/MyDrive/FYP/Velocity/3_2.json already exists.
Processing video: 4_2
Velocity features file /content/drive/MyDrive/FYP/Velocity/4_2.json already exists.
Processing video: 16_1
Velocity features file /content/drive/MyDrive/FYP/Velocity/16_1.json already exists.
Processing video: 15_1
Velocity features file /content/drive/MyDrive/FYP/Velocity/15_1.json already exists.
Processing video: 14_1
Velocity features file /content/drive/MyDrive/FYP/Velocity/14_1.json already exists.
Processing video: 13_1
Velocity features file /content/drive/MyDrive/FYP/Velocity/13_1.json already exists.
Processing video: 12_3
Velocity features file /content/drive/MyDrive/FYP/Velocity/12_3.json already exists.
Processing video: 11_3
Velocity feat

In [ ]:
# Machine learning pipeline classifier code
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import KFold, GroupKFold, GridSearchCV, LeaveOneGroupOut
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.svm import SVC
# from cuml.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, precision_score, recall_score, f1_score, make_scorer
from mpl_toolkits.mplot3d import Axes3D
from sklearn.pipeline import Pipeline
import joblib, json



def plot_class_separation_3d(X, Y, title):
    class_names = ['doing own work', 'passing paper', "looking at other's work"]
    num_classes = len(class_names)

    # Create a color map
    cmap = plt.get_cmap('viridis', num_classes)

    # Create a figure and a 3D axis
    fig = plt.figure(figsize=(9, 6))
    ax = fig.add_subplot(111, projection='3d')

    # Plot each class with its own color
    for i in range(num_classes):
        indices = Y == i
        ax.scatter(X[indices, 0], X[indices, 1],
                   color=cmap(i), label=class_names[i], alpha=0.7)

    # Create a legend with class names
    ax.legend(title='Class')
    # ax.set_aspect('auto')
    # Set titles and labels
    # ax.set_title(title)
    ax.set_xlabel('LD1')
    ax.set_ylabel('LD2')
    ax.set_zlabel('LD3')

    plt.show()


def gridSearch(X, Y, group_ids):

    num_videos, num_frames, num_features = X.shape

    # Step 4: Divide into smaller video samples using a sliding window approach
    all_video_samples = []
    window_size = 30
    step_size = 2
    all_y_pred = []
    all_y_true = []


    for video_idx in range(num_videos):
        video_sample = X[video_idx]
        video_samples = []

        for start in range(0, num_frames - window_size + 1, step_size):
            end = start + window_size
            sample = video_sample[start:end]
            video_samples.append(sample)
        all_video_samples.append(video_samples)

    all_video_samples = np.array(all_video_samples)
    num_samples_per_video = all_video_samples.shape[1]
    num_total_samples = num_videos * num_samples_per_video

    all_video_samples = all_video_samples.reshape(num_total_samples, window_size * num_features)


    print("all videos sample shape", all_video_samples.shape)
    Y = np.repeat(Y, num_samples_per_video)
    group_ids = np.repeat(group_ids, num_samples_per_video)

    # Define the parameter grid for SVM
    param_grid = {
    'svm__C': [0.01, 0.1, 1, 10, 100],                # Include default value 1.0
    'svm__gamma': [1e-3, 1e-4, 1e-5, 1e-1, 1e-2, 'scale', 'auto'], # Include default values 'scale' and 'auto'
    'svm__kernel': ['linear', 'poly', 'rbf', 'sigmoid'] # Try different kernels
    }


    # Create a pipeline that includes scaling, LDA, and SVM
    pipeline = Pipeline([
        ('scaler', StandardScaler()),        # Feature scaling
        ('lda', LDA(n_components=2)),        # Linear Discriminant Analysis
        ('svm', SVC(kernel="rbf"))           # SVM with RBF kernel
    ])

    # Use LeaveOneGroupOut cross-validation
    logo = LeaveOneGroupOut()

    # Define scoring metrics
    scoring = {
        'accuracy': 'accuracy',
        'precision_macro': make_scorer(precision_score, average='macro', zero_division=0),
        'recall_macro': make_scorer(recall_score, average='macro', zero_division=0),
        'f1_macro': make_scorer(f1_score, average='macro', zero_division=0)
    }

    # Set up GridSearchCV with the pipeline, parameter grid, and cross-validation scheme
    grid_search = GridSearchCV(pipeline, param_grid, cv=logo, scoring=scoring, refit=False)

    # Fit the grid search to the data
    grid_search.fit(all_video_samples, Y, groups=group_ids)

    best_index = grid_search.cv_results_['rank_test_accuracy'].argmin()

    best_params = grid_search.cv_results_['params'][best_index]

    best_accuracy = grid_search.cv_results_['mean_test_accuracy'][best_index]
    # Get the best parameters
    print(f'Best parameters: {best_params}')

    # Get the best metrics
    print(f'Best accuracy: {best_accuracy:.4f}')

    return best_params

def classificationWithBestParam(X, Y, group_ids, best_params):

    num_videos, num_frames, num_features = X.shape

    # Step 4: Divide into smaller video samples using a sliding window approach
    all_video_samples = []
    all_energy_features_video_samples = []
    window_size = 30
    step_size = 5
    all_y_pred = []
    all_y_true = []


    for video_idx in range(num_videos):
        video_sample = X[video_idx]
        video_samples = []

        for start in range(0, num_frames - window_size + 1, step_size):
            end = start + window_size
            sample = video_sample[start:end]
            video_samples.append(sample)
        all_video_samples.append(video_samples)

    all_video_samples = np.array(all_video_samples)
    num_samples_per_video = all_video_samples.shape[1]
    num_total_samples = num_videos * num_samples_per_video

    print("all videos samples before", all_video_samples.shape)

    all_video_samples = all_video_samples.reshape(num_total_samples, window_size * num_features)


    print("all videos sample shape", all_video_samples.shape)
    Y = np.repeat(Y, num_samples_per_video)
    group_ids = np.repeat(group_ids, num_samples_per_video)

    scaler = MinMaxScaler()
    logo = LeaveOneGroupOut()
    lda = LDA(n_components=2)

    accuracies = []
    precisions = []
    recalls = []
    f1_scores = []

    save_dir = '/content/drive/MyDrive/FYP/ML_training'
    os.makedirs(save_dir, exist_ok=True)
    fold = 0

    for train_index, test_index in logo.split(all_video_samples, Y, groups=group_ids):
        X_train, X_test = all_video_samples[train_index], all_video_samples[test_index]
        Y_train, Y_test = Y[train_index], Y[test_index]

        print("X_train shape", X_train.shape)
        print("X_test shape", X_test.shape)

        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

        X_train_lda = lda.fit_transform(X_train, Y_train)
        X_test_lda = lda.transform(X_test)

        best_svm = SVC(kernel = best_params['svm__kernel'], C=best_params['svm__C'], gamma= best_params['svm__gamma'])
        best_svm.fit(X_train_lda, Y_train)

        plot_class_separation_3d(X_train_lda, Y_train, 'Training Data')
        plot_class_separation_3d(X_test_lda, Y_test, 'Testing Data')

        Y_pred = best_svm.predict(X_test_lda)
        accuracy = accuracy_score(Y_test, Y_pred)
        precision = precision_score(Y_test, Y_pred, average='macro',zero_division=0)
        recall = recall_score(Y_test, Y_pred, average='macro')
        f1 = f1_score(Y_test, Y_pred, average='macro')

        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        f1_scores.append(f1)

        all_y_pred.extend(Y_pred)
        all_y_true.extend(Y_test)
        print(f'Fold accuracy: {accuracy * 100:.2f}%')

         # Save trained models using joblib
        lda_model_path = os.path.join(save_dir, f"lda_fold_{fold}.joblib")
        svm_model_path = os.path.join(save_dir, f"svm_fold_{fold}.joblib")
        scalar_model_path = os.path.join(save_dir, f"scaler_fold_{fold}.joblib")

        joblib.dump(lda, lda_model_path)
        joblib.dump(best_svm, svm_model_path)
        joblib.dump(scaler, scalar_model_path)

        fold += 1


    # Compute and display confusion matrix
    cm = confusion_matrix(all_y_true, all_y_pred, labels=np.unique(Y))

    # Normalize the confusion matrix to show percentages
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    class_names = ['doing own work', 'passing paper', "looking at other's work"]
    # Plot confusion matrix with accuracy percentages
    disp = ConfusionMatrixDisplay(confusion_matrix=cm_normalized, display_labels=class_names)
    fig, ax = plt.subplots(figsize=(8, 5))

    disp.plot(ax=ax, cmap=plt.cm.Blues, values_format=".2f")
    # Set aspect ratio to be equal
    ax.set_aspect('auto')
    plt.title("Confusion Matrix")
    plt.show()

    print(f'Mean precision: {np.mean(precisions):.4f}')
    print(f'Mean recall: {np.mean(recalls):.4f}')
    print(f'Mean SVM accuracy: {np.mean(accuracies) * 100:.2f}%')
    print(f"fl score: {np.mean(f1_scores)}")


# Read extracted features for each video
def reading_features(hof_path, dist_path, angle_path):

    with open(dist_path, 'r') as f:
        distance_features_data = json.load(f)

    with open(hof_path, 'r') as f:
        hof_features_data = json.load(f)

    with open(angle_path, 'r') as f:
        angle_features_data = json.load(f)

    distance_features = np.array(distance_features_data['intra_inter_distance_mapping'])
    # distance_features = np.array(distance_features_data['velocity'])
    angle_features = np.array(angle_features_data['intra_inter_angle_mapping'])
    hof_features = np.array(hof_features_data['hof'])

    # intra_inter_angle_mapping
    combined_features = np.concatenate((
                                        distance_features,
                                        angle_features,
                                        hof_features,
                                        ), axis=1)

    return combined_features

if __name__ == "__main__":
    # Assuming you have a method to get your data as a DataFrame
    video_df = get_ground_truth()

    X = []
    Y = []
    group_ids = []

    for video, video_row in video_df.iterrows():
        combined_features  = reading_features(video_row['hof_path'], video_row['dist_path'], video_row['angle_path'])
        X.append(combined_features)
        Y.append(video_row['action'])
        group_ids.append(video_row['fold'])

    X = np.array(X)
    Y = np.array(Y)

    # best_param = gridSearch(X, Y, group_ids)
    best_param = {'svm__C': 0.1, 'svm__gamma': 0.1, 'svm__kernel': 'sigmoid'}
    classificationWithBestParam(X, Y, group_ids, best_param)


# Mean precision: 0.7607
# Mean recall: 0.7187
# Mean SVM accuracy: 71.43%
# fl score: 0.7148465509648211

# Mean precision: 0.8413
# Mean recall: 0.8267
# Mean SVM accuracy: 81.59%
# fl score: 0.8187946736089304

# Best parameters: {'svm__C': 0.1, 'svm__gamma': 0.1, 'svm__kernel': 'sigmoid'}


# 420, 210  (30, 5) 76.51
# 420, 210 (40, 3) 78.89


In [ ]:
# {'hidden_dim': 256, 'num_layers': 1, 'lr': 0.001, 'bidirectional': True}
# Best Accuracy: 91.82%  (50, 1)

# {'hidden_dim': 128, 'num_layers': 1, 'lr': 0.001, 'bidirectional': True}
# Best Accuracy: 90.71% (40, 2)

class BiLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers, bidirectional=False):
        super(BiLSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.bidirectional = bidirectional

        # LSTM Layer (Bi-directional if specified)
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, bidirectional=bidirectional)

        # Output Layer
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)

        # ReLU activation and Dropout for regularization
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        # LSTM Layer - Get output and hidden state (we don't need the hidden state here)
        lstm_out, _ = self.lstm(x)  # LSTM output shape: (batch_size, seq_len, hidden_dim * 2)

        # If bidirectional, we take the output of the last timestep for each direction
        if self.bidirectional:
            # We take the concatenation of the forward and backward hidden states at the last time step
            final_hidden_state = torch.cat((lstm_out[:, -1, :self.hidden_dim], lstm_out[:, 0, self.hidden_dim:]), dim=-1)

            # final_hidden_state = lstm_out[:, -1, :self.hidden_dim] + lstm_out[:, 0, self.hidden_dim:]
        else:
            # If not bidirectional, just take the last timestep output
            final_hidden_state = lstm_out[:, -1, :]

        # Apply Dropout, then fully connected layer
        output = self.fc(self.dropout(self.relu(final_hidden_state)))

        return output


# Read extracted features for each video
def reading_features(hof_path, dist_path, angle_path, ltp_path, velocity_path):
    with open(dist_path, 'r') as f:
        distance_features = json.load(f)

    with open(hof_path, 'r') as f:
        hof_features = json.load(f)

    with open(angle_path, 'r') as f:
        angle_features = json.load(f)

    with open(ltp_path, 'r') as f:
        ltp_features = json.load(f)

    with open(velocity_path, 'r') as f:
        vel_features = json.load(f)

    # print("distance", np.array(distance_features['intra_inter_distance_mapping']).shape)
    # print("angle", np.array(angle_features['intra_inter_angle_mapping']).shape)
    # print("hof", np.array(hof_features['hof']).shape)
    # print("ltp", np.array(ltp_features['ltp']).shape)
    # print("vel", np.array(vel_features['velocity']).shape)

    combined_features = np.concatenate((
        np.array(distance_features['intra_inter_distance_mapping']),
        np.array(angle_features['intra_inter_angle_mapping']),
        np.array(vel_features['velocity']),
        np.array(ltp_features['ltp']),
        np.array(hof_features['hof']),
    ), axis=1)

    return combined_features


def get_samples(X, Y, group_ids):
    num_videos, num_frames, num_features = X.shape
    num_classes = len(np.unique(Y))
    # Step 4: Divide into smaller video samples using a sliding window approach
    window_size = 40
    step_size = 2
    all_video_samples = []

    for video_idx in range(num_videos):
        video_sample = X[video_idx]
        video_samples = [video_sample[start:start + window_size] for start in range(0, num_frames - window_size + 1, step_size)]
        all_video_samples.append(video_samples)


    X = np.array(all_video_samples)
    num_samples_per_video = X.shape[1]
    X = X.reshape(-1, window_size, num_features)
    Y = np.repeat(Y, num_samples_per_video)
    group_ids = np.repeat(group_ids, num_samples_per_video)

    return X, Y, group_ids

In [ ]:

# Read extracted features for each video
def reading_features(hof_path, dist_path, angle_path, ltp_path, velocity_path):

    if os.path.exists(dist_path):
      with open(dist_path, 'r') as f:
          distance_features = json.load(f)
      np.save(dist_path.split('.')[0] + '.npy', np.array(distance_features['intra_inter_distance_mapping']))
      os.remove(dist_path)

    if os.path.exists(hof_path):
      with open(hof_path, 'r') as f:
          hof_features = json.load(f)
      np.save(hof_path.split('.')[0] + '.npy', np.array(hof_features['hof']))
      os.remove(hof_path)

    if os.path.exists(angle_path):
      with open(angle_path, 'r') as f:
          angle_features = json.load(f)
      np.save(angle_path.split('.')[0] + '.npy', np.array(angle_features['intra_inter_angle_mapping']))
      os.remove(angle_path)

    if os.path.exists(ltp_path):
      with open(ltp_path, 'r') as f:
          ltp_features = json.load(f)
      np.save(ltp_path.split('.')[0] + '.npy', np.array(ltp_features['ltp']))
      os.remove(ltp_path)


    if os.path.exists(velocity_path):
      with open(velocity_path, 'r') as f:
          vel_features = json.load(f)
      np.save(velocity_path.split('.')[0] + '.npy', np.array(vel_features['velocity']))
      os.remove(velocity_path)


    # geo_features = np.load(geo_path)
    # print("distance", np.array(intra_inter_distance_features['intra_inter_distance_mapping']).shape)
    # print("angle", np.array(angle_features['intra_inter_angle_mapping']).shape)
    # print("hof", np.array(hof_features['hof']).shape)
    # print("ltp", np.array(ltp_features['ltp']).shape)
    # print("hog", np.array(hog_features['hog']).shape)
    # print("vel", np.array(vel_features['velocity']).shape)

    # combined_features = np.concatenate((
    #     np.array(distance_features['distance']),
    #     geo_features,
    #     # np.array(angle_features['angle']),
    #     # np.array(vel_features['velocity']),
    #     # np.array(hof_features['hof']),
    #     # np.array(hog_features['hog']),

    #     np.array(ltp_features['ltp']),
    #     np.array(hof_features['hof']),
    #     # np.array(lpq_features['lpq'])
    # ), axis=1)

    # return combined_features
#  ["dist", "angle", "vel", "hog", "hof"]

if __name__ == "__main__":
    # Assuming you have a method to get your data as a DataFrame
    video_df = get_ground_truth()

    X = []
    Y = []
    group_ids = []

    for video, video_row in video_df.iterrows():
        reading_features(video_row['fixed_hof_path'], video_row['dist_path'], video_row['angle_path'],
                      video_row['ltp_path'], video_row['vel_path'])
        print("processed", video)


processed 1_2
processed 2_2
processed 3_2
processed 4_2
processed 16_1
processed 15_1
processed 14_1
processed 13_1
processed 12_3
processed 11_3
processed 9_3
processed 7_3
processed 17_1
processed 18_2
processed 19_3
processed 20_1
processed 23_1
processed 26_1
processed 29_1
processed 32_2
processed 33_1
processed 34_3
processed 35_2
processed 36_1
processed 37_3
processed 38_2
processed 40_3
processed 39_1
processed 41_3
processed 43_2
processed 42_1
processed 44_3
processed 45_1
processed 46_2
processed 22_3
processed 21_2
processed 24_2
processed 25_3
processed 27_2
processed 28_3
processed 30_2
processed 31_3
processed 47_3
processed 48_2
processed 49_1
processed 50_3
processed 51_2
processed 52_1
processed 53_3
processed 54_2
processed 55_1
processed 57_3
processed 56_2
processed 58_1
processed 59_3
processed 60_2
processed 61_1
processed 62_3
processed 63_2
processed 64_1
processed 65_3
processed 66_2
processed 67_1
processed 68_3
processed 69_1
processed 71_2
processed 72_1
p

In [ ]:

# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# {'hidden_dim': 128, 'num_layers': 1, 'lr': 0.001, 'bidirectional': True}
# Best Accuracy: 90.71% (40, 2)

# best params hidden_dim=64, num_layers=2, lr=1e-3, epochs=50
def classificationWithBiLSTM(X, Y, group_ids, hidden_dim=128, num_layers=1, lr=1e-3, bidirectional=True, epochs=100):
    X, Y, group_ids = get_samples(X, Y, group_ids)
    num_videos, window_size, num_features = X.shape
    print("num_features", num_features)
    num_classes = len(np.unique(Y))

    all_y_true = []
    all_y_pred = []

    save_dir = '/content/drive/MyDrive/FYP/DL_training'
    os.makedirs(save_dir, exist_ok=True)

    SCALERS = {
      "dist": MinMaxScaler(),
      "angle": MinMaxScaler(),
      "hof": StandardScaler(),
      "vel": RobustScaler(),
      "ltp": MinMaxScaler()
    }

    selected_features_len = {
      "dist": 12,
      "angle": 15,
      "hof": 18,
      "vel": 14,
      "ltp": 512
    }

    selected_features = ["dist", "angle", "vel", "ltp", "hof"]
    logo = LeaveOneGroupOut()
    accuracies, precisions, recalls, f1_scores = [], [], [], []

    for train_idx, test_idx in logo.split(X, Y, groups=group_ids):
        X_train, X_test = X[train_idx], X[test_idx]
        Y_train, Y_test = Y[train_idx], Y[test_idx]

        # print("X_train shape", X_train.shape)
        # print("X_test shape", X_test.shape)
        # print("Y_train shape", Y_train.shape)
        # print("Y_test shape", Y_test.shape)

        prev_feat_length = 0
        standard_X_train, standard_X_test = None, None

        for feature in selected_features:
            X_train_subset = X_train[:, :, prev_feat_length:prev_feat_length + selected_features_len[feature]].reshape(X_train.shape[0], -1)
            X_test_subset = X_test[:, :, prev_feat_length:prev_feat_length + selected_features_len[feature]].reshape(X_test.shape[0], -1)

            # X_train_subset = X_train[:, :, prev_feat_length:prev_feat_length + selected_features_len[feature]].reshape(X_train.shape[0], -1)
            # X_test_subset = X_test[:, :, prev_feat_length:prev_feat_length + selected_features_len[feature]].reshape(X_test.shape[0], -1)

            # print("feature", feature)
            # print("X_train", X_train_subset.shape)
            # print("Y_train", Y_train.shape)
            print("feature", feature)
            print("X_train_subset shape", X_test_subset.shape)
            print("X_test_subset shape", X_test_subset.shape)
            scaler = SCALERS[feature]
            X_train_subset = scaler.fit_transform(X_train_subset)
            X_test_subset = scaler.transform(X_test_subset)

            scalar_model_path = os.path.join(save_dir, f"scaler_{feature}_fold_{len(accuracies)}.joblib")
            joblib.dump(scaler, scalar_model_path)

            X_train_subset = X_train_subset.reshape(X_train.shape[0], window_size, -1)
            X_test_subset = X_test_subset.reshape(X_test.shape[0], window_size, -1)

            if standard_X_train is None:
                standard_X_train = X_train_subset
                standard_X_test = X_test_subset
            else:
                standard_X_train = np.concatenate((standard_X_train, X_train_subset), axis=2)
                standard_X_test = np.concatenate((standard_X_test, X_test_subset), axis=2)

            prev_feat_length += selected_features_len[feature]

        X_train = standard_X_train
        X_test = standard_X_test

        # Convert data to PyTorch tensors
        X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
        Y_train_tensor = torch.tensor(Y_train, dtype=torch.long).to(device)
        X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
        Y_test_tensor = torch.tensor(Y_test, dtype=torch.long).to(device)

        train_data = Data.TensorDataset(X_train_tensor, Y_train_tensor)
        test_data = Data.TensorDataset(X_test_tensor, Y_test_tensor)

        train_loader = Data.DataLoader(dataset=train_data, batch_size=16, shuffle=True)
        test_loader = Data.DataLoader(dataset=test_data, batch_size=16, shuffle=False)

        best_accuracy = 0.0  # Track best accuracy
        best_model_path = f"{save_dir}/best_model_fold_{len(accuracies)}.pth"  # Unique model filename for each fold

        model = BiLSTM(num_features, hidden_dim, num_classes, num_layers, bidirectional).to(device)

        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=lr)

        # Learning Rate Scheduler
        # scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.1)

        scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)

        # best_loss = float('inf')
        best_accuracy = 0.0  # Track best accuracy
        best_model_path = f"{save_dir}/best_model_fold_{len(accuracies)}.pth"  # Unique model filename for each fold

        # Training loop
        for epoch in range(epochs):
            model.train()
            for inputs, labels in train_loader:
                optimizer.zero_grad()
                inputs = inputs.to(device)
                labels = labels.to(device)
                outputs = model(inputs)

                # print("inputs shape", inputs.shape)
                # print("labels shape", labels.shape)
                # print("outputs shape", outputs.shape)
                # print("labels shape", labels.shape)

                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

            # Validation (after each epoch)
            model.eval()
            y_pred_fold, y_true_fold = [], []
            val_loss = 0.0
            with torch.no_grad():
                for inputs, labels in test_loader:
                    inputs = inputs.to(device)
                    labels = labels.to(device)
                    outputs = model(inputs)

                    # print("outputs shape", outputs.shape)
                    # print("labels shape", labels.shape)

                    loss = criterion(outputs, labels)
                    val_loss += loss.item()

                    _, preds = torch.max(outputs, 1)
                    y_pred_fold.extend(preds.cpu().numpy())
                    y_true_fold.extend(labels.cpu().numpy())
            val_loss /= len(test_loader)

            # Compute validation accuracy
            accuracy = accuracy_score(y_true_fold, y_pred_fold)

            if accuracy > best_accuracy:  # Save model if accuracy improves
                best_accuracy = accuracy
                torch.save(model.state_dict(), best_model_path)
                print(f"Epoch {epoch+1}: Validation Loss: {val_loss}")

            # scheduler.step(val_loss)  # Update learning rate
            scheduler.step()

        print(f'Best accuracy for fold {len(accuracies)}: {best_accuracy * 100:.2f}% (Model saved)')

        # Testing
        model.load_state_dict(torch.load(best_model_path, weights_only=True))
        model.eval()

        y_pred_fold, y_true_fold = [], []
        with torch.no_grad():
            for inputs, labels in test_loader:
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                y_pred_fold.extend(preds.detach().cpu().numpy())  # Ensure moving tensors to CPU before converting to numpy
                y_true_fold.extend(labels.detach().cpu().numpy())

        # Metrics
        accuracy = accuracy_score(y_true_fold, y_pred_fold)
        precision = precision_score(y_true_fold, y_pred_fold, average='macro', zero_division=0)
        recall = recall_score(y_true_fold, y_pred_fold, average='macro')
        f1 = f1_score(y_true_fold, y_pred_fold, average='macro')

        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        f1_scores.append(f1)

        all_y_pred.extend(y_pred_fold)
        all_y_true.extend(y_true_fold)

        print(f'Fold accuracy: {accuracy * 100:.2f}%')

    # Confusion matrix visualization
    cm = confusion_matrix(all_y_true, all_y_pred, labels=np.unique(Y))
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    class_names = ['doing own work', 'passing paper', "looking at other's work"]

    disp = ConfusionMatrixDisplay(confusion_matrix=cm_normalized, display_labels=class_names)
    fig, ax = plt.subplots(figsize=(8, 5))
    disp.plot(ax=ax, cmap=plt.cm.Blues, values_format=".2f")
    plt.title("Confusion Matrix")
    plt.show()

    print(f'Mean precision: {np.mean(precisions):.4f}')
    print(f'Mean recall: {np.mean(recalls):.4f}')
    print(f'Mean BiLSTM accuracy: {np.mean(accuracies) * 100:.2f}%')
    print(f"Mean F1-score: {np.mean(f1_scores)}")


if __name__ == "__main__":
    # Assuming you have a method to get your data as a DataFrame
    video_df = get_ground_truth()

    X = []
    Y = []
    group_ids = []

    for video, video_row in video_df.iterrows():
        combined_feat  = reading_features(video_row['hof_path'], video_row['dist_path'], video_row['angle_path'],
                      video_row['ltp_path'], video_row['fixed_velocity_path'])

        # print("combined_features", combined_feat.shape)
        X.append(combined_feat)
        Y.append(video_row['action'])
        group_ids.append(video_row['fold'])

    X = np.array(X)
    Y = np.array(Y)

    classificationWithBiLSTM(X, Y, group_ids)



In [ ]:

# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# 32, 64, 128, 256]
# Main classification function with grid search
def gridSearchWithBiLSTM(X, Y, group_ids, hidden_dims=[128, 256], num_layers_list=[1, 2], lrs=[1e-3], bidirectional_list=[False, True], epochs=100):
    X, Y, group_ids = get_samples(X, Y, group_ids)
    num_videos, window_size, num_features = X.shape
    num_classes = len(np.unique(Y))

    overall_best_accuracy = 0
    best_params = {}
    save_dir = "/content/drive/MyDrive/FYP/DL_training"

    SCALERS = {
      "dist": MinMaxScaler(),
      "angle": MinMaxScaler(),
      "hof": StandardScaler(),
      "vel": RobustScaler(),
      "ltp": MinMaxScaler(),
    }

    selected_features_len = {
      "dist": 12,
      "angle": 15,
      "hof": 18,
      "vel": 14,
      "ltp": 512
      # "efficientNet": 1408
    }

    selected_features = ["dist", "angle", "vel", "ltp", "hof"]

    # Loop over all combinations of parameters
    for hidden_dim in hidden_dims:
        for num_layers in num_layers_list:
            for lr in lrs:
                for bidirectional in bidirectional_list:

                    print(f"Training with hidden_dim={hidden_dim}, num_layers={num_layers}, lr={lr}, bidirectional={bidirectional}")

                    logo = LeaveOneGroupOut()
                    accuracies = []

                    for train_idx, test_idx in logo.split(X, Y, groups=group_ids):

                        X_train, X_test = X[train_idx], X[test_idx]
                        Y_train, Y_test = Y[train_idx], Y[test_idx]

                        print("X_test shape", X_test.shape)

                        prev_feat_length = 0
                        standard_X_train, standard_X_test = None, None

                        for feature in selected_features:
                            X_train_subset = X_train[:, :, prev_feat_length:prev_feat_length + selected_features_len[feature]].reshape(X_train.shape[0], -1)
                            X_test_subset = X_test[:, :, prev_feat_length:prev_feat_length + selected_features_len[feature]].reshape(X_test.shape[0], -1)

                            # print("feature", feature)
                            # print("X_train", X_train_subset.shape)
                            # print("Y_train", Y_train.shape)

                            scaler = SCALERS[feature]
                            X_train_subset = scaler.fit_transform(X_train_subset)
                            X_test_subset = scaler.transform(X_test_subset)

                            X_train_subset = X_train_subset.reshape(X_train.shape[0], window_size, -1)
                            X_test_subset = X_test_subset.reshape(X_test.shape[0], window_size, -1)

                            if standard_X_train is None:
                                standard_X_train = X_train_subset
                                standard_X_test = X_test_subset
                            else:
                                standard_X_train = np.concatenate((standard_X_train, X_train_subset), axis=2)
                                standard_X_test = np.concatenate((standard_X_test, X_test_subset), axis=2)

                            prev_feat_length += selected_features_len[feature]

                        X_train = standard_X_train
                        X_test = standard_X_test

                        print("X_train", X_train.shape)
                        print("Y_train", Y_train.shape)

                        # Convert data to PyTorch tensors
                        X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
                        Y_train_tensor = torch.tensor(Y_train, dtype=torch.long).to(device)
                        X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
                        Y_test_tensor = torch.tensor(Y_test, dtype=torch.long).to(device)

                        train_data = Data.TensorDataset(X_train_tensor, Y_train_tensor)
                        test_data = Data.TensorDataset(X_test_tensor, Y_test_tensor)

                        train_loader = Data.DataLoader(dataset=train_data, batch_size=32, shuffle=True)
                        test_loader = Data.DataLoader(dataset=test_data, batch_size=32, shuffle=False)

                        if bidirectional:
                            epochs = 100
                            # lr=0.0001

                        model = BiLSTM(num_features, hidden_dim, num_classes, num_layers, bidirectional).to(device)

                        criterion = nn.CrossEntropyLoss()
                        optimizer = optim.Adam(model.parameters(), lr=lr)

                        # Learning Rate Scheduler
                        # scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.1)

                        scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)

                        # best_loss = float('inf')
                        best_accuracy = 0.0  # Track best accuracy
                        best_model_path = f"{save_dir}/best_model_fold_{len(accuracies)}.pth"  # Unique model filename for each fold

                        # Training loop
                        for epoch in range(epochs):
                            model.train()
                            for inputs, labels in train_loader:
                                optimizer.zero_grad()
                                inputs = inputs.to(device)
                                labels = labels.to(device)
                                outputs = model(inputs)

                                # print("inputs shape", inputs.shape)
                                # print("labels shape", labels.shape)
                                # print("outputs shape", outputs.shape)
                                # print("labels shape", labels.shape)

                                loss = criterion(outputs, labels)
                                loss.backward()
                                optimizer.step()

                            # Validation (after each epoch)
                            model.eval()
                            y_pred_fold, y_true_fold = [], []
                            val_loss = 0.0
                            with torch.no_grad():
                                for inputs, labels in test_loader:
                                    inputs = inputs.to(device)
                                    labels = labels.to(device)
                                    outputs = model(inputs)

                                    # print("outputs shape", outputs.shape)
                                    # print("labels shape", labels.shape)

                                    loss = criterion(outputs, labels)
                                    val_loss += loss.item()

                                    _, preds = torch.max(outputs, 1)
                                    y_pred_fold.extend(preds.cpu().numpy())
                                    y_true_fold.extend(labels.cpu().numpy())
                            val_loss /= len(test_loader)

                            # Compute validation accuracy
                            accuracy = accuracy_score(y_true_fold, y_pred_fold)

                            if accuracy > best_accuracy:  # Save model if accuracy improves
                                best_accuracy = accuracy
                                torch.save(model.state_dict(), best_model_path)

                            # scheduler.step(val_loss)  # Update learning rate
                            scheduler.step()
                            # print(f"Epoch {epoch+1}: Validation Loss: {val_loss}")

                        accuracies.append(best_accuracy)

                    mean_accuracy = np.mean(accuracies)

                    print("mean accuracy", mean_accuracy)

                    # Update best parameters if current combination is better
                    if mean_accuracy > overall_best_accuracy:
                        overall_best_accuracy = mean_accuracy
                        best_params = {
                            "hidden_dim": hidden_dim,
                            "num_layers": num_layers,
                            "lr": lr,
                            "bidirectional": bidirectional
                        }

    # Output the best parameters
    print("Best Parameters:")
    print(best_params)
    print(f"Best Accuracy: {overall_best_accuracy * 100:.2f}%")


if __name__ == "__main__":
    # Assuming you have a method to get your data as a DataFrame
    video_df = get_ground_truth()

    X = []
    Y = []
    group_ids = []

    for video, video_row in video_df.iterrows():
        combined_feat  = reading_features(video_row['hof_path'], video_row['dist_path'], video_row['angle_path'],
                      video_row['ltp_path'], video_row['fixed_velocity_path'])

        # print("combined_features", combined_feat.shape)
        X.append(combined_feat)
        Y.append(video_row['action'])
        group_ids.append(video_row['fold'])

    X = np.array(X)
    Y = np.array(Y)

    gridSearchWithBiLSTM(X, Y, group_ids)

# dist (60, 12)
# angle (60, 15)
# hog (60, 3780)
# vel (60, 28)
# ltp (60, 512)
# hof (60, 18)


Training with hidden_dim=128, num_layers=1, lr=0.001, bidirectional=False
X_test shape (330, 40, 571)
X_train (660, 40, 571)
Y_train (660,)
X_test shape (330, 40, 571)
X_train (660, 40, 571)
Y_train (660,)
X_test shape (330, 40, 571)
X_train (660, 40, 571)
Y_train (660,)
mean accuracy 0.907070707070707
Training with hidden_dim=128, num_layers=1, lr=0.001, bidirectional=True
X_test shape (330, 40, 571)
X_train (660, 40, 571)
Y_train (660,)
X_test shape (330, 40, 571)
X_train (660, 40, 571)
Y_train (660,)
X_test shape (330, 40, 571)
X_train (660, 40, 571)
Y_train (660,)
mean accuracy 0.8868686868686867
Training with hidden_dim=128, num_layers=2, lr=0.001, bidirectional=False
X_test shape (330, 40, 571)
X_train (660, 40, 571)
Y_train (660,)
X_test shape (330, 40, 571)
X_train (660, 40, 571)
Y_train (660,)
X_test shape (330, 40, 571)
X_train (660, 40, 571)
Y_train (660,)
mean accuracy 0.8777777777777778
Training with hidden_dim=128, num_layers=2, lr=0.001, bidirectional=True
X_test shape (